In [4]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import os

# Load data FIRST
csv_file = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
"
if os.path.exists(csv_file):
    spacex_df = pd.read_csv(csv_file)
    print(f"✅ Loaded {len(spacex_df)} rows from {csv_file}")
else:
    print(f"❌ {csv_file} not found!")
    exit(1)

# Calculate payload range
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()
print(f"Payload range: {min_payload}-{max_payload} kg")

# ✅ FIXED: Define site_options BEFORE layout
sites = spacex_df['Launch Site'].unique().tolist()
site_options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in sites]
print(f"Available sites: {sites}")

# Initialize app
app = dash.Dash(__name__)

# Layout - NOW site_options is defined
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Launch Site:"),
        dcc.Dropdown(
            id='site-dropdown',
            options=site_options,  # ✅ Now properly defined
            value='ALL',
            placeholder="Select a Launch Site...",
            searchable=True
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Payload Range (kg):"),
        dcc.RangeSlider(
            id='payload-slider',
            min=0, max=10000, step=1000,
            value=[min_payload, max_payload],
            marks={
                0: '0', 2500: '2.5k', 5000: '5k', 
                7500: '7.5k', 10000: '10k'
            }
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    html.Br(),
    
    html.Div(id='success-pie-chart', style={'width': '48%', 'display': 'inline-block'}),
    html.Div(id='success-payload-scatter-chart', style={'width': '48%', 'display': 'inline-block'})
])

# Callback 1: Pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='children'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', 
                    title='Total Success Launches By Site',
                    color_discrete_map={'1': 'green', '0': 'red'})
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, values='class', names='class', 
                    title=f"Success Launches for {entered_site}",
                    color_discrete_map={'1': 'green', '0': 'red'})
    return dcc.Graph(figure=fig)

# Callback 2: Scatter plot
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='children'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_plot(entered_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]
    
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                       color='Booster Version Category',
                       title='Payload vs Success (All Sites)',
                       hover_data=['Flight Number'])
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(site_df, x='Payload Mass (kg)', y='class',
                       color='Booster Version Category',
                       title=f"Payload vs Success ({entered_site})",
                       hover_data=['Flight Number'])
    return dcc.Graph(figure=fig)

if __name__ == '__main__':
    app.run(debug=True, port=8060)


SyntaxError: unterminated string literal (detected at line 8) (2440837966.py, line 8)